In [ ]:
#importing libraries

import tensorflow as tf
import os
import pandas as pd
import cv2
import numpy as np
from matplotlib import pyplot as plt
import matplotlib

In [ ]:
# creating function to get numpy arrays of images and corresponding labels

def generate_data(use):
    use_data = os.path.join("..", "archive", use)
    
    use_pos = os.listdir(os.path.join(use_data, "Positive"))
    use_neg = os.listdir(os.path.join(use_data, "Negative"))
    pos_imgs = [cv2.imread(os.path.join(use_data, "Positive", x)) for x in use_pos]
    neg_imgs = [cv2.imread(os.path.join(use_data, "Negative", x)) for x in use_neg]
    use_imgs = []
    labels = []

    for img in pos_imgs:
        use_imgs.append(img)
        labels.append(1)

    for img in neg_imgs:
        use_imgs.append(img)
        labels.append(0)
        
    use_imgs = np.array(use_imgs) / 255.0
    labels = np.array(labels)
        
    return (use_imgs, labels)

In [ ]:
#creating train and validation numpy arrays of images and labels

train_imgs, train_labels = generate_data("train")
valid_imgs, valid_labels = generate_data("valid")

In [ ]:
#shuffling the train and validation datasets so that poitive and negative labelled images intermix randomly

def shuffle_together(a, b):
    a = list(a); b = list(b)
    dset = []; X = []; Y = []
    if (len(a) == len(b)):
        length = len(a)
    else:
        return (0, 0)
    c = zip(a, b)
    
    for item in c:
        dset.append(item)
    dset = np.array(dset)
    np.random.shuffle(dset)
    
    for item in dset:
        X.append(item[0])
        Y.append(item[1])
        
    X = np.array(X); Y = np.array(Y)
    
    return (X, Y)

train_imgs, train_labels = shuffle_together(train_imgs, train_labels)
valid_imgs, valid_labels = shuffle_together(valid_imgs, valid_labels)

In [ ]:
# importing tensorflow for creating neural network

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Activation

In [ ]:
# Making custom function for image augmentation

def augment(image, label):
    # sizing to a standard size
    new_height = 224
    new_width = 224
    image = tf.image.resize(image, (new_height, new_width))
    
    # changing color-scheme randomly
    if tf.random.uniform((), minval = 0, maxval = 1) <= 0.1:
        image = tf.tile(tf.image.rgb_to_grayscale(image), [1, 1, 3])
        
    # randomly allotting brightness level
    image = tf.image.random_brightness(image, max_delta = 0.5)
    
    # flipping horizontally - 50% of the times
    image = tf.image.random_flip_left_right(image)
    
    # rotating
    image = tf.image.rot90(image, k = int(tf.random.uniform((), minval = 0, maxval = 4)))
    
    return np.array(image), label

In [ ]:
# This will generate an array containing augmented images from the train set supplied as input variable

def augmented_set(use_imgs, labels, k = 2):
    use_set = zip(list(use_imgs), list(labels))
    new_set = []
    
    for point in use_set:
        for i in range(k):
            new_img, new_label = augment(point[0], point[1])
            new_set.append((new_img, new_label))
    
    new_imgs = [point[0] for point in new_set]
    new_labels = [point[1] for point in new_set]
    
    return np.array(new_imgs, dtype = object), np.array(new_labels)

In [ ]:
# getting the augmented dataset from the train set created above

dtrain_imgs, dtrain_labels = augmented_set(train_imgs, train_labels, 4)

In [ ]:
# creating neural network

model = Sequential([
    Conv2D(32, (3, 3), activation = "relu", input_shape = (224, 224, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(32, (3, 3), activation = "relu"),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation = "relu"),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation = "relu"),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation = "relu"),
    Dense(1, activation = "sigmoid")
])

In [ ]:
model.compile(loss = "binary_crossentropy", optimizer = "Adam", metrics = ["accuracy"])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 109, 109, 32)      9248      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 54, 54, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 52, 52, 64)        18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 26, 26, 64)      

In [ ]:
dtrain_labels = np.asarray(dtrain_labels).astype("float32")
dtrain_imgs = np.asarray(dtrain_imgs).astype("float32")

In [ ]:
# training on train dataset

model.fit(dtrain_imgs, dtrain_labels, batch_size = 16, epochs = 12, validation_data = (valid_imgs, valid_labels))

Epoch 1/12
149/149 [==============================] - 17s 110ms/step - loss: 0.6930 - accuracy: 0.5566 - val_loss: 0.6912 - val_accuracy: 0.5000
Epoch 2/12
149/149 [==============================] - 16s 109ms/step - loss: 0.6920 - accuracy: 0.4975 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/12
149/149 [==============================] - 15s 101ms/step - loss: 0.6849 - accuracy: 0.5365 - val_loss: 0.6917 - val_accuracy: 0.5000
Epoch 4/12
149/149 [==============================] - 16s 108ms/step - loss: 0.6407 - accuracy: 0.6086 - val_loss: 0.6917 - val_accuracy: 0.5150
Epoch 5/12
149/149 [==============================] - 18s 118ms/step - loss: 0.6691 - accuracy: 0.5772 - val_loss: 0.6360 - val_accuracy: 0.8650
Epoch 6/12
149/149 [==============================] - 18s 120ms/step - loss: 0.5447 - accuracy: 0.7303 - val_loss: 0.4686 - val_accuracy: 0.8350
Epoch 7/12
149/149 [==============================] - 18s 121ms/step - loss: 0.3961 - accuracy: 0.8356 - val_loss: 0.2101 - val_ac

In [ ]:
test_imgs, test_labels = generate_data("test")
res = model.evaluate(test_imgs, test_labels, batch_size = 16)
print("test loss, test accuracy: ", res)

13/13 [==============================] - 0s 28ms/step - loss: 0.0307 - accuracy: 1.0000
test loss, test accuracy:  [0.030655747279524803, 1.0]


In [ ]:
pred_labels = []

for test_img in test_imgs:
    test_img = test_img.reshape((1, ) + test_img.shape)
    prediction = model.predict(test_img)
    
    if (prediction[0][0] > 0.5):
        pred_labels.append(1)
    else:
        pred_labels.append(0)

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


In [ ]:
conf = np.array(tf.math.confusion_matrix(np.array(test_labels), np.array(pred_labels)))

In [ ]:
conf

array([[100,   0],
       [  0, 100]])

In [ ]:
precision_score = conf[0][0] / (conf[0][0] + conf[0][1])
recall = conf[0][0] / (conf[0][0] + conf[1][0])

f1_score = (2 * precision_score * recall) / (precision_score + recall)

print("precision: ", precision_score, " recall: ", recall, " F1: ", f1_score)

precision:  1.0  recall:  1.0  F1:  1.0


In [ ]:
# to predict on any image of any dimensions and having crack at any location in the image

def predict_any_image(img_path, model):
    img = cv2.imread(img_path) / 255.0
    img_ht = img.shape[0]
    img_wt = img.shape[1]

    # this variable will become 1 when crack is found in image
    done = 0
    
    # if dimensions of image are less than (224, 224) then reshaping to increase size to 224
    if (img_ht < 224):
        img = cv2.resize(img, (224, img.shape[1]))
    if (img_wt < 224):
        img = cv2.resize(img, (img.shape[0], 224))
        
    # if image is now of exact dimension as required, simply predict using the model and print result, then exit the function
    if (img.shape == (224, 224, 3)):
        img = img.reshape((1, ) + img.shape)
        pred = model.predict(img, verbose = 3)
        if (pred[0][0] > 0.5):
            print("Cracked")
            return
        else:
            print("Not-Cracked")
            return
    
    # if any dimension of image is now > 336 then begin looping with a window of 224 x 224 and check if crack found
    if (img.shape[0] > 336 or img.shape[1] > 336):
        
        # "l" variable is used to shift the window along height of image and "k" to shift along width
        l = k = 0

        # loop till we don't reach the bottom of the image
        while (l + 224 <= img.shape[0]):
            k = 0

            # at a particular value of l i.e at a particular height of the window loop on variable "k" to shift window along width of image
            while (k + 224 <= img.shape[1]):

                # creating a window named img_sub of size 224 x 224
                img_sub = img[l: l + 224, k: k + 224]

                # reshaping window to the dimensions as required by the neural network
                img_sub = img_sub.reshape((1, ) + img_sub.shape)

                # making predictions
                pred = model.predict(img_sub, verbose = 3)
                if (pred[0][0] > 0.5):
                    print("Cracked")
                    done = 1
                    return
                else:
                    # if not found then update "k" to shift window by 112 units along the width of the image
                    k += 112
            
            # when full width checked of image at particular height, shift window downwards by 112 units
            l += 112
    else:
        # if image is not that big then simple resizing wouldn't harm the image quality much so simply resize and predict
        img = cv2.resize(img, (224, 224, 3))
        
        pred = model.predict(img, verbose = 3)
        if (pred[0][0] > 0.5):
            print("Cracked")
            return
        else:
            print("Not-Cracked")
            return
    
    # done is still 0 i.e. crack is not found so, print "Not-Cracked" and exit function
    if (done == 0):
        print("Not-Cracked")
        return